In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
df=pd.read_csv("C:/Users/USER/OneDrive/Masaüstü/Denemeler/costa-rican-household/train.csv")

In [5]:
df.head()

,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,...,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target
0,ID_279628684,190000.0,0,3,0,1,1,0,NaN,0,...,100,1849,1,100,0,1.000000,0.0,100.0,1849,4
1,ID_f29eb3ddd,135000.0,0,4,0,1,1,1,1.0,0,...,144,4489,1,144,0,1.000000,64.0,144.0,4489,4
2,ID_68de51c94,NaN,0,8,0,1,1,0,NaN,0,...,121,8464,1,0,0,0.250000,64.0,121.0,8464,4
3,ID_d671db89c,180000.0,0,5,0,1,1,1,1.0,0,...,81,289,16,121,4,1.777778,1.0,121.0,289,4
4,ID_d56d6f5f5,180000.0,0,5,0,1,1,1,1.0,0,...,121,1369,16,121,4,1.777778,1.0,121.0,1369,4


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9557 entries, 0 to 9556
Columns: 143 entries, Id to Target
dtypes: float64(8), int64(130), object(5)
memory usage: 10.4+ MB


In [7]:
object_columns=df.select_dtypes(include=["object"]).columns
for col in object_columns:
    df[col]=pd.factorize(df[col])[0]+1

In [22]:
na_columns=df.columns[df.isna().sum() > 0].tolist()

In [13]:
for i in na_columns:
    print(f"{i}: {df[i].median()}")

v2a1: 130000.0
v18q1: 1.0
rez_esc: 0.0
meaneduc: 9.0
SQBmeaned: 81.0


In [14]:
for i in na_columns:
    print(f"{i}: {df[i].mean()}")

v2a1: 165231.6069707082
v18q1: 1.4040632054176072
rez_esc: 0.4597912829957029
meaneduc: 9.231522935701422
SQBmeaned: 102.58886699019682


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9557 entries, 0 to 9556
Columns: 143 entries, Id to Target
dtypes: float64(8), int64(135)
memory usage: 10.4 MB


# Filling with median is a better way

In [23]:
df[na_columns]=df[na_columns].apply(lambda x: x.fillna(x.median()))

In [25]:
## Data has availability to increase the features

train_set=df

In [26]:
df['rent_per_adult'] = df['v2a1']/df['hogar_adul']
df['rent_per_person'] = df['v2a1']/df['hhsize']

df['overcrowding_room_and_bedroom'] = (df['hacdor'] + df['hacapo'])/2

df['no_appliances'] = df['refrig'] + df['computer'] + df['television']

df['r4h1_percent_in_male'] = df['r4h1'] / df['r4h3']
df['r4m1_percent_in_female'] = df['r4m1'] / df['r4m3']
df['r4h1_percent_in_total'] = df['r4h1'] / df['hhsize']
df['r4m1_percent_in_total'] = df['r4m1'] / df['hhsize']
df['r4t1_percent_in_total'] = df['r4t1'] / df['hhsize']

df['adult'] = df['hogar_adul'] - df['hogar_mayor']
df['dependency_count'] = train_set['hogar_nin'] + df['hogar_mayor']
df['dependency'] = df['dependency_count'] / df['adult']
df['child_percent'] = df['hogar_nin']/df['hogar_total']
df['elder_percent'] = df['hogar_mayor']/df['hogar_total']
df['adult_percent'] = df['hogar_adul']/df['hogar_total']

df['rent_per_bedroom'] = df['v2a1']/df['bedrooms']
df['adults_per_bedroom'] = df['adult']/df['bedrooms']
df['child_per_bedroom'] = df['hogar_nin']/df['bedrooms']
df['male_per_bedroom'] = df['r4h3']/df['bedrooms']

df['female_per_bedroom'] = df['r4m3']/df['bedrooms']
df['bedrooms_per_person_household'] = df['hhsize']/df['bedrooms']

df['tablet_per_person_household'] = df['v18q1']/df['hhsize']
df['phone_per_person_household'] = df['qmobilephone']/df['hhsize']

df['age_12_19'] = df['hogar_nin'] - df['r4t1']

df['rent_per_room'] = df['v2a1']/df['rooms']
df['bedroom_per_room'] = df['bedrooms']/df['rooms']
df['elder_per_room'] = df['hogar_mayor']/df['rooms']
df['adults_per_room'] = df['adult']/df['rooms']
df['child_per_room'] = df['hogar_nin']/df['rooms']
df['male_per_room'] = df['r4h3']/df['rooms']
df['female_per_room'] = df['r4m3']/df['rooms']
df['room_per_person_household'] = df['hhsize']/df['rooms']
df['escolari_age'] = df['escolari']/df['age']

df['rez_esc_escolari'] = df['rez_esc']/df['escolari']
df['rez_esc_r4t1'] = df['rez_esc']/df['r4t1']
df['rez_esc_r4t2'] = df['rez_esc']/df['r4t2']
df['rez_esc_r4t3'] = df['rez_esc']/df['r4t3']
df['rez_esc_age'] = df['rez_esc']/df['age']

# We've small portion of data so there is no need for PCA or something else

In [93]:
from sklearn import svm
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_validate,cross_val_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
X=df.drop("Target", axis=1)
y=df["Target"]

In [94]:
X_test, X_train, y_test, y_train = train_test_split(X,y, test_size=0.2)

In [95]:
models={
    "SVM": svm.SVC(),
    "SGD Classifier":SGDClassifier() 
}

In [96]:
from sklearn.impute import SimpleImputer

# NaN ve Sonsuz değerleri temizle
X_train = np.where(np.isfinite(X_train), X_train, np.nan)
X_test = np.where(np.isfinite(X_test), X_test, np.nan)

# Eksik değerleri median ile doldur
imputer = SimpleImputer(strategy="median")
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)


In [97]:
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import precision_score, recall_score, f1_score

# Model listesi - NearestNeighbors() çıkarıldı çünkü sınıflandırıcı değil!
models = {
    "SVM": svm.SVC(),
    "SGD Classifier": SGDClassifier()
}

def fit_and_score(models, X_train, X_test, y_train, y_test):
    model_scores1 = {}

    for name, model in models.items():
        model.fit(X_train, y_train)  # Modeli eğitiyoruz
        y_pred = model.predict(X_test)  # Tahmin yapıyoruz

        accuracy = model.score(X_test, y_test)
        precision = precision_score(y_test, y_pred, average="weighted")
        recall = recall_score(y_test, y_pred, average="weighted")
        f1 = f1_score(y_test, y_pred, average="weighted")

        model_scores1[name] = {
            "Accuracy": accuracy,
            "Precision": precision,
            "Recall": recall,
            "F1 Score": f1
        }

    return model_scores1

In [99]:
# Model skorlarını hesapla
model_scores1 = fit_and_score(models, X_train, X_test, y_train, y_test)

C:\Users\USER\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [100]:
model_scores1

{'SVM': {'Accuracy': 0.6295618051013735,
  'Precision': 0.39634806644249976,
  'Recall': 0.6295618051013735,
  'F1 Score': 0.4864474181976097},
 'SGD Classifier': {'Accuracy': 0.6115107913669064,
  'Precision': 0.49249698242415585,
  'Recall': 0.6115107913669064,
  'F1 Score': 0.5169818584800929}}

## I'll Use Another Classifiers soon (xgboost, lgbm etc.)